# Extract AOD for each OpenAQ measurement

## Import library

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import requests
from pprint import pp
from tqdm.notebook import tqdm
import requests
import time
from zkyhaxpy import io_tools, pd_tools, gis_tools
from datetime import datetime, timedelta
import os

## Prepare GeoDataframe of OpenAQ data

In [ ]:
if not os.path.exists('../data/gdf_openaq.gpkg'):
    df_openaq = pd.read_csv('../data/openaqi_thailand_pm25_data.csv', sep=',' )
    df_openaq['measurement_id'] = df_openaq.index

    df_openaq['geometry'] = 'POINT (' + df_openaq['long'].astype(str) + ' ' + df_openaq['lat'].astype(str) + ')'
    gdf_openaq = gis_tools.df_to_gdf(df_openaq, 'geometry')

    gdf_openaq['datetime'] = gdf_openaq['date_utc'].apply(lambda date_utc: pd.Timestamp(date_utc).tz_convert("UTC"))
    gdf_openaq['year'] = gdf_openaq['date_utc'].str.slice(0, 4).astype(int)
    gdf_openaq['month'] = gdf_openaq['date_utc'].str.slice(5, 7).astype(int)
    del(gdf_openaq['date_utc'])
    gdf_openaq = gdf_openaq.set_index('measurement_id')
    gdf_openaq.to_file('../data/gdf_openaq.gpkg')
    print('gdf_openaq has been saved.')
else:
    gdf_openaq = gpd.read_file('../data/gdf_openaq.gpkg')
    print('gdf_openaq has been loaded.')
if gdf_openaq.index.name is None:
    gdf_openaq = gdf_openaq.set_index('measurement_id')

In [ ]:
dir_extracted_aod_root = r'../../../data/aod/extracted/aod_055'
df_files_aod = io_tools.get_list_files(dir_extracted_aod_root, '.tif$', return_df=True)
df_tmp = df_files_aod['file_path'].str.split('\\', expand=True)
df_files_aod['tile_id'] = df_tmp[df_tmp.columns[-3]]
df_files_aod['date'] = df_files_aod['file_nm'].str.slice(8, 18)
df_files_aod['time'] = df_files_aod['file_nm'].str.slice(19, 21) + ':' + df_files_aod['file_nm'].str.slice(21, 23)
df_files_aod['datetime'] = df_files_aod['date'] + 'T' + df_files_aod['time'] + ':00+00:00'
df_files_aod['datetime'] = df_files_aod['datetime'].apply(lambda datetime: pd.Timestamp(datetime).tz_convert("UTC"))
df_files_aod['year'] = df_files_aod['datetime'].dt.year
df_files_aod

# DEV

In [ ]:
#Filter only image covering Thailand and time matched with OpenAQ data
list_tile_id = ['h27v07', 'h27v08', 'h28v07', 'h28v08']
df_files_aod = df_files_aod[df_files_aod['tile_id'].isin(list_tile_id)]
df_files_aod = df_files_aod[df_files_aod['year'] >= 2019]
df_files_aod

In [ ]:
MAX_DIFF_HOURS = 1
list_df_extracted_aod055 = []
list_error = []
df_files_aod = df_files_aod.sample(frac=1.0)
pbar = tqdm(df_files_aod.iterrows(), total=len(df_files_aod))
for s_idx, s_row in pbar:
    
    min_datetime = s_row.datetime - timedelta(hours=MAX_DIFF_HOURS)
    max_datetime = s_row.datetime + timedelta(hours=MAX_DIFF_HOURS)
    gdf_openaq_tmp = gdf_openaq.copy()
    gdf_openaq_tmp = gdf_openaq_tmp[gdf_openaq_tmp.datetime.between(min_datetime, max_datetime)]
    gdf_openaq_tmp = gdf_openaq_tmp[gdf_openaq_tmp.datetime.between(min_datetime, max_datetime)]
    
    with rasterio.open(s_row.file_path) as ds:
        crs_aod = ds.crs
        
    if len(gdf_openaq_tmp) > 0:
        try:
            df_extracted_aod055_curr = gis_tools.extract_pixval_single_file(
                in_s_polygon=gdf_openaq_tmp['geometry'].to_crs(crs_aod),
                in_raster_path=s_row.file_path,
                in_list_out_col_nm=['aod_055'],
                in_list_target_raster_band_id=[1],
                nodata_val=-28672
                )
            df_extracted_aod055_curr['tile_id'] = s_row.tile_id
            df_extracted_aod055_curr['aod_datetime'] = s_row.datetime
            list_df_extracted_aod055.append(df_extracted_aod055_curr)            
        except ValueError:
            list_error.append({
                'gdf_openaq_tmp':gdf_openaq_tmp,
                'raster_path':s_row.file_path
            })

    pbar.set_description(f'success={len(list_df_extracted_aod055)} / error={len(list_error)} ')
    

In [ ]:
df_extracted_aod055 = pd.concat(list_df_extracted_aod055)
df_extracted_aod055.to_csv(r'../data/df_extracted_aod055.csv', index=False)

In [ ]:
(df_extracted_aod055['aod_055']==-28672).mean()

In [ ]:
gdf_openaq_remain = gdf_openaq[~(gdf_openaq.index.isin(df_extracted_aod055['measurement_id'].values))].copy()

In [ ]:
df_files_aod['datetime'] = df_files_aod['datetime'].apply(lambda datetime: pd.Timestamp(datetime).tz_convert("UTC"))

In [ ]:
df_files_aod['datetime'] 

In [ ]:
df_extracted_aod055['aod_datetime'].dt.hour.hist(bins=24)

In [ ]:
gdf_openaq_remain['datetime'].dt.hour.hist(bins=24)

In [ ]:
df_extracted_aod055_curr

In [ ]:
profile

In [ ]:
gdf_openaq_tmp.crs